<a href="https://colab.research.google.com/github/scarabinoalbano/03MIAR_04_A_2025-26_Algoritmos-de-Optimizacion/blob/main/AlbanoScarabino_ProyectoDeProgramacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algoritmos de optimización - Seminario**<br>
Nombre y Apellidos: Albano Scarabino <br>
Colab: https://colab.research.google.com/drive/1wAZzWmNN3LBZpulwaiPIveJe-T_gMkxA?usp=sharing <br>
Url: https://github.com/scarabinoalbano/03MIAR_04_A_2025-26_Algoritmos-de-Optimizacion/blob/main/AlbanoScarabino_ProyectoDeProgramacion.ipynb

---
# **Problema 1: Organizar sesiones de doblaje**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:
- Número de actores: 10
- Número de tomas: 30
- Actores/Tomas: https://bit.ly/36D8IuK
   - 1 indica que el actor participa en la toma
   - 0 en caso contrario

In [ ]:
matriz_enunciado = [
        [1,1,1,1,1,0,0,0,0,0],  # Toma 1
        [0,0,1,1,1,0,0,0,0,0],  # Toma 2
        [0,1,0,0,1,0,1,0,0,0],  # Toma 3
        [1,1,0,0,0,0,1,1,0,0],  # Toma 4
        [0,1,0,1,0,0,0,1,0,0],  # Toma 5
        [1,1,0,1,1,0,0,0,0,0],  # Toma 6
        [1,1,0,1,1,0,0,0,0,0],  # Toma 7
        [1,1,0,0,0,1,0,0,0,0],  # Toma 8
        [1,1,0,1,0,0,0,0,0,0],  # Toma 9
        [1,1,0,0,0,1,0,0,1,0],  # Toma 10
        [1,1,1,0,1,0,0,1,0,0],  # Toma 11
        [1,1,1,1,0,1,0,0,0,0],  # Toma 12
        [1,0,0,1,1,0,0,0,0,0],  # Toma 13
        [1,0,1,0,0,1,0,0,0,0],  # Toma 14
        [1,1,0,0,0,0,1,0,0,0],  # Toma 15
        [0,0,0,1,0,0,0,0,0,1],  # Toma 16
        [1,0,1,0,0,0,0,0,0,0],  # Toma 17
        [0,0,1,0,0,1,0,0,0,0],  # Toma 18
        [1,0,1,0,0,0,0,0,0,0],  # Toma 19
        [1,0,1,1,1,0,0,0,0,0],  # Toma 20
        [0,0,0,0,0,1,0,1,0,0],  # Toma 21
        [1,1,1,1,0,0,0,0,0,0],  # Toma 22
        [1,0,1,0,0,0,0,0,0,0],  # Toma 23
        [0,0,1,0,0,1,0,0,0,0],  # Toma 24
        [1,1,0,1,0,0,0,0,0,1],  # Toma 25
        [1,0,1,0,1,0,0,0,1,0],  # Toma 26
        [0,0,0,1,1,0,0,0,0,0],  # Toma 27
        [1,0,0,1,0,0,0,0,0,0],  # Toma 28
        [1,0,0,0,1,1,0,0,0,0],  # Toma 29
        [1,0,0,1,0,0,0,0,0,0]   # Toma 30
]
max_tomas_por_dia_enunciado = 6

---
# **Algoritmo con "Solver pulp" para luego comparar con "Fuerza Bruta" y "Busqueda Tabú".**

In [ ]:
import time
import pulp

def optimizar_horario_doblaje(matriz_actores_tomas, max_tomas_por_dia):
    """
    Optimiza el horario de doblaje minimizando el costo total de actores.
    """
    t0 = time.time()
    num_tomas = len(matriz_actores_tomas)
    num_actores = len(matriz_actores_tomas[0])

    # Estimación del número mínimo de días necesarios
    min_dias = max(1, (num_tomas + max_tomas_por_dia - 1) // max_tomas_por_dia)
    max_dias = min(num_tomas, min_dias + 5)

    print(f"Buscando solución entre {min_dias} y {max_dias} días...")

    # Búsqueda iterativa empezando por el mínimo número de días
    for num_dias in range(min_dias, max_dias + 1):
        print(f"Probando con {num_dias} días...")

        # Crear el problema de optimización
        prob = pulp.LpProblem("Doblaje_Horario", pulp.LpMinimize)

        # Variables de decisión usando diccionarios
        x = {}
        for t in range(num_tomas):
            for d in range(num_dias):
                x[t, d] = pulp.LpVariable(f"x_{t}_{d}", cat='Binary')

        y = {}
        for a in range(num_actores):
            for d in range(num_dias):
                y[a, d] = pulp.LpVariable(f"y_{a}_{d}", cat='Binary')

        # Función objetivo
        prob += pulp.lpSum([y[a, d] for a in range(num_actores) for d in range(num_dias)])

        # Restricciones
        # 1. Cada toma debe grabarse exactamente una vez
        for t in range(num_tomas):
            prob += pulp.lpSum([x[t, d] for d in range(num_dias)]) == 1

        # 2. Máximo número de tomas por día
        for d in range(num_dias):
            prob += pulp.lpSum([x[t, d] for t in range(num_tomas)]) <= max_tomas_por_dia

        # 3. Si un actor participa en una toma que se graba en un día, el actor debe trabajar ese día
        for a in range(num_actores):
            for d in range(num_dias):
                for t in range(num_tomas):
                    if matriz_actores_tomas[t][a] == 1:
                        prob += y[a, d] >= x[t, d]

        # Resolver el problema
        prob.solve(pulp.PULP_CBC_CMD(msg=0, timeLimit=30))

        # Verificar si se encontró una solución factible
        if prob.status == pulp.LpStatusOptimal:
            costo = pulp.value(prob.objective)
            print(f"Solución encontrada con {num_dias} días. Costo: {costo}")
            print(f"Tiempo total: {time.time() - t0:.2f} segundos")
            return {'num_dias': num_dias, 'costo': costo, 'x': x, 'y': y}
        else:
            print(f"No se encontró solución factible con {num_dias} días")

    print("No se encontró solución factible")
    return None

def mostrar_resultados(solucion, matriz_actores_tomas, max_tomas_por_dia):
    """Muestra los resultados de la optimización de forma clara"""

    if solucion is None:
        print("No se encontró solución factible")
        return

    num_tomas = len(matriz_actores_tomas)
    num_actores = len(matriz_actores_tomas[0])
    num_dias = solucion['num_dias']
    x = solucion['x']
    y = solucion['y']

    print("\n=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE ===\n")

    # Mostrar planificación día por día
    for d in range(num_dias):
        print(f"DIA: {d+1}")

        # Tomas del día
        tomas_dia = [t for t in range(num_tomas) if pulp.value(x[t, d]) == 1]
        print(f"  Tomas: {[t+1 for t in tomas_dia]}")

        # Actores del día
        actores_dia = [a for a in range(num_actores) if pulp.value(y[a, d]) == 1]
        print(f"  Actores necesarios: {[a+1 for a in actores_dia]}")

        # Costo del día
        print(f"  Costo del dia: {len(actores_dia)}\n")

    print(f"COSTO TOTAL: {int(solucion['costo'])}")
    print(f"DIAS TOTALES: {num_dias}\n")

def ejecutar_solver_pulp(matriz_actores_tomas, max_tomas_por_dia):
    """
    Función principal que ejecuta toda la optimización del horario de doblaje
    """
    print("="*60)
    print("ORGANIZAR SESIONES DE DOBLAJE - SOLVER PULP")
    print("="*60)

    # Ejecutar la optimización
    solucion = optimizar_horario_doblaje(matriz_actores_tomas, max_tomas_por_dia)

    # Mostrar los resultados
    mostrar_resultados(solucion, matriz_actores_tomas, max_tomas_por_dia)

In [ ]:
ejecutar_solver_pulp(matriz_enunciado, max_tomas_por_dia_enunciado)

ORGANIZAR SESIONES DE DOBLAJE - SOLVER PULP
Buscando solución entre 5 y 10 días...
Probando con 5 días...
Solución encontrada con 5 días. Costo: 27.0
Tiempo total: 30.08 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE ===

DIA: 1
  Tomas: [14, 17, 18, 19, 23, 24]
  Actores necesarios: [1, 3, 6]
  Costo del dia: 3

DIA: 2
  Tomas: [3, 4, 8, 11, 15, 21]
  Actores necesarios: [1, 2, 3, 5, 6, 7, 8]
  Costo del dia: 7

DIA: 3
  Tomas: [5, 9, 16, 25, 28, 30]
  Actores necesarios: [1, 2, 4, 8, 10]
  Costo del dia: 5

DIA: 4
  Tomas: [2, 6, 7, 13, 20, 27]
  Actores necesarios: [1, 2, 3, 4, 5]
  Costo del dia: 5

DIA: 5
  Tomas: [1, 10, 12, 22, 26, 29]
  Actores necesarios: [1, 2, 3, 4, 5, 6, 9]
  Costo del dia: 7

COSTO TOTAL: 27
DIAS TOTALES: 5



---
# **¿Cúantas posibilidades hay sin tener en cuenta las restricciones?**

## Fórmula con restricción de tomas por día (k)

Sean:

- `(n)`: número total de tomas  
- `(k)`: máximo de tomas por día  
- `(d)`: número de días  
- `(R(n, d, k))`: número de formas de particionar `(n)` tomas en `(d)` días, con hasta `(k)` tomas por día (considerando orden)

#### Fórmula general (con orden de los días)

$$
P(n, k) = \sum_{d = \lceil n / k \rceil}^{n} R(n, d, k)
$$

#### Fórmula general (sin orden de los días)

$$
P(n, k) = \sum_{d = \lceil n / k \rceil}^{n} \frac{R(n, d, k)}{d!}
$$

#### Donde:

$$
R(n, d, k) = \sum_{i=1}^{\min(k, n)} R(n - i, d - 1, k)
$$

## Fórmula sin restricción de tomas por día (k → ∞)

Cuando no hay restricción de cuántas tomas pueden hacerse por día, el número total de particiones posibles del conjunto de `n` tomas es dado por el **n-ésimo número de Bell** `(B_n)`:

#### Fórmula general:

$$
P(n, \infty) = B_n
$$

donde `(B_n)` es el número de Bell, que cuenta las formas de particionar un conjunto de `n` elementos en subconjuntos no vacíos (sin importar el orden de los subconjuntos).

#### Recurrencia:

$$
B_0 = 1 \\
B_{n+1} = \sum_{k=0}^{n} \binom{n}{k} B_k
$$

In [ ]:
import math

def bell_number(n):
    B = [0] * (n + 1)
    B[0] = 1
    for i in range(1, n + 1):
        B[i] = sum(math.comb(i - 1, k) * B[k] for k in range(i))
    return B[n]

# Ejemplo para n = 30
print(bell_number(30))

846749014511809332450147


---
# **¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?**

## Ejemplo de cálculo para n = 30, k = 6

#### Fórmula general (sin orden de los días)

$$
P(30, 6) = \sum_{d = \lceil 30/6 \rceil}^{30} \frac{R(30, d, 6)}{d!}
$$

Es decir:

- Mínimo número de días:
$$
\lceil \tfrac{30}{6} \rceil = 5
$$

- Máximo número de días: 30
- Entonces se calcula:

$$
P(30, 6) = \frac{R(30, 5, 6)}{5!} + \frac{R(30, 6, 6)}{6!} + \cdots + \frac{R(30, 30, 6)}{30!}
$$


In [ ]:
import math
from functools import lru_cache

@lru_cache(maxsize=None)
def R(n, d, k):
    """
    Calcula la cantidad de particiones ordenadas de `n` tomas en `d` días, donde cada día tiene entre 1 y `k` tomas como máximo.
    """
    if n == 0 and d == 0:
        return 1
    if n < 0 or d == 0 or n > d * k:
        return 0

    total = 0
    for i in range(1, min(k, n - d + 1) + 1):
        total += math.comb(n, i) * R(n - i, d - 1, k)
    return total

def total_posibilidades_sin_orden(n, k):
    """
    Calcula el número total de particiones válidas de `n` tomas en días, donde cada día tiene entre 1 y `k` tomas, sin importar el orden de los días.
    """
    d_min = math.ceil(n / k)
    total = 0
    for d in range(d_min, n + 1):
        total += R(n, d, k) // math.factorial(d)
    return total

# Definir rangos para n y k
rangos_n = list(range(4, 31))  # Del 4 al 30 inclusive
rangos_k = [3, 4, 5, 6, 7, 8]

print("Análisis de iteraciones para diferentes valores de n y k")
print("=" * 60)
print(f"{'n':>3} | {'k':>3} | {'Iteraciones':>15}")
print("-" * 30)

# Doble bucle para barrer todos los valores
for n in rangos_n:
    for k in rangos_k:
            total_iteraciones = total_posibilidades_sin_orden(n, k)
            print(f"{n:>3} | {k:>3} | {total_iteraciones:>15,}")
    print("-" * 30)  # Separador entre diferentes valores de n

Análisis de iteraciones para diferentes valores de n y k
  n |   k |     Iteraciones
------------------------------
  4 |   3 |              14
  4 |   4 |              15
  4 |   5 |              15
  4 |   6 |              15
  4 |   7 |              15
  4 |   8 |              15
------------------------------
  5 |   3 |              46
  5 |   4 |              51
  5 |   5 |              52
  5 |   6 |              52
  5 |   7 |              52
  5 |   8 |              52
------------------------------
  6 |   3 |             166
  6 |   4 |             196
  6 |   5 |             202
  6 |   6 |             203
  6 |   7 |             203
  6 |   8 |             203
------------------------------
  7 |   3 |             652
  7 |   4 |             827
  7 |   5 |             869
  7 |   6 |             876
  7 |   7 |             877
  7 |   8 |             877
------------------------------
  8 |   3 |           2,780
  8 |   4 |           3,795
  8 |   5 |           4,075
  8 

In [ ]:
# Es computacionalmente imposible resolver n=30, k=6 con fuerza bruta.
total_iteraciones = total_posibilidades_sin_orden(len(matriz_enunciado), max_tomas_por_dia_enunciado)
print(f"\nSe necesitan {total_iteraciones} iteraciones.")


Se necesitan 726391948970868949621309 iteraciones.


---
# **¿Cuál es la función objetivo?**

Minimizar la cantidad total de actores involucrados a lo largo de todos los días:

$$
\text{Minimizar: } f(x) = \sum_{d=1}^{D} |A_d|
$$

Donde:

- $D$: número total de días  
- $A_d$: conjunto de actores únicos necesarios en el día $d$  
- $|A_d|$: número de actores únicos en el día $d$ (cardinalidad del conjunto $A_d$)

In [ ]:
def funcion_objetivo(planificacion, matriz_actores_tomas):
    """
    Función objetivo: calcular el costo total sin optimizaciones
    """
    costo_total = 0

    for dia in planificacion:
        # Contar actores únicos del día
        actores_del_dia = []
        for toma in dia:
            for actor in range(len(matriz_actores_tomas[0])):
                if matriz_actores_tomas[toma][actor] == 1:
                    if actor not in actores_del_dia:
                        actores_del_dia.append(actor)

        costo_total += len(actores_del_dia)

    return costo_total

---
# **¿Es un problema de maximización o de minimización?**

Es definitivamente un problema de minimización donde se busca la planificación con el menor costo total de actores-día.

Sujeto a:
- Cada toma se asigna a exactamente un día
- Máximo 6 tomas por día
- Todos los actores necesarios están disponibles cada día

---
# **Diseña un algoritmo para resolver el problema por fuerza bruta.**

In [ ]:
import time
import itertools

def generar_todas_asignaciones(num_tomas, max_dias):
    """
    Genera todas las posibles asignaciones de tomas a días.
    Cada toma se asigna a un día específico (0, 1, 2, ..., max_dias-1)
    """
    # Generar todas las combinaciones posibles donde cada toma puede ir a cualquier día
    for asignacion in itertools.product(range(max_dias), repeat=num_tomas):
        yield asignacion

def asignacion_a_planificacion(asignacion, max_tomas_por_dia):
    """
    Convierte una asignación (tupla de días para cada toma) a una planificación válida
    Retorna None si la asignación viola las restricciones
    """
    planificacion = [[] for _ in range(max(asignacion) + 1)]

    # Asignar cada toma a su día correspondiente
    for toma, dia in enumerate(asignacion):
        planificacion[dia].append(toma)

    # Verificar que ningún día exceda el máximo de tomas permitidas
    for dia in planificacion:
        if len(dia) > max_tomas_por_dia:
            return None

    # Filtrar días vacíos
    planificacion_filtrada = [dia for dia in planificacion if dia]

    return planificacion_filtrada

def funcion_objetivo(planificacion, matriz_actores_tomas):
    """
    Función objetivo: calcular el costo total sin optimizaciones
    """
    costo_total = 0

    for dia in planificacion:
        # Contar actores únicos del día
        actores_del_dia = []
        for toma in dia:
            for actor in range(len(matriz_actores_tomas[0])):
                if matriz_actores_tomas[toma][actor] == 1:
                    if actor not in actores_del_dia:
                        actores_del_dia.append(actor)

        costo_total += len(actores_del_dia)

    return costo_total

def fuerza_bruta(matriz_actores_tomas, max_tomas_por_dia):
    """
    Algoritmo de fuerza bruta sin optimizaciones
    """
    num_tomas = len(matriz_actores_tomas)
    max_dias_posibles = num_tomas  # En el peor caso, cada toma en un día diferente

    mejor_costo = float('inf')
    mejor_planificacion = None
    iteraciones = 0
    iteraciones_validas = 0

    print("\nGenerando todas las asignaciones posibles...")

    # Probar todas las asignaciones posibles
    for asignacion in generar_todas_asignaciones(num_tomas, max_dias_posibles):
        iteraciones += 1

        # Convertir asignación a planificación
        planificacion = asignacion_a_planificacion(asignacion, max_tomas_por_dia)

        # Si la planificación es válida, evaluar su costo
        if planificacion is not None:
            iteraciones_validas += 1

            # Calcular costo usando función objetivo
            costo_total = funcion_objetivo(planificacion, matriz_actores_tomas)

            # Actualizar mejor solución
            if costo_total < mejor_costo:
                mejor_costo = costo_total
                mejor_planificacion = [dia[:] for dia in planificacion]
                print(f"Nueva mejor solucion encontrada - Costo: {mejor_costo}, Dias: {len(planificacion)}, Asignacion: {asignacion}")

        # Mostrar progreso cada cierto número de iteraciones
        if iteraciones % 100000 == 0:
            print(f"Iteracion: {iteraciones}, Validas: {iteraciones_validas}, Mejor costo: {mejor_costo}, Asignacion actual: {asignacion}")

    print(f"Iteraciones Totales: {iteraciones}, Iteraciones Validas: {iteraciones_validas}, Mejor costo: {mejor_costo}")
    return mejor_planificacion

def mostrar_planificacion(planificacion, matriz_actores_tomas):
    """
    Muestra la planificación de manera detallada (igual que el original)
    """
    if not planificacion:
        print("No se encontro planificacion")
        return

    print("\n=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===")

    costo_total = 0
    num_actores = len(matriz_actores_tomas[0])

    for i, dia in enumerate(planificacion, 1):
        print(f"\nDIA {i}:")
        print(f"Tomas: {[t+1 for t in dia]}")  # +1 para mostrar tomas desde 1

        # Calcular actores únicos del día
        actores_del_dia = []
        for toma in dia:
            for actor in range(num_actores):
                if matriz_actores_tomas[toma][actor] == 1:
                    if (actor + 1) not in actores_del_dia:
                        actores_del_dia.append(actor + 1)  # +1 para mostrar actores desde 1

        actores_del_dia.sort()
        print(f"Actores necesarios: {actores_del_dia}")
        print(f"Costo del dia: {len(actores_del_dia)}")

        costo_total += len(actores_del_dia)

    print(f"\nCOSTO TOTAL: {costo_total}")
    print(f"DIAS TOTALES: {len(planificacion)}\n")

def ejecutar_algoritmo_fuerza_bruta(matriz_actores_tomas, max_tomas_por_dia):
    """
    Función principal para resolver el problema de doblaje con algoritmo de fuerza bruta
    """
    print("="*60)
    print("ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA")
    print("="*60)

    # Ejecutar algoritmo de fuerza bruta
    tiempo_inicio = time.time()
    mejor_planificacion = fuerza_bruta(matriz_actores_tomas, max_tomas_por_dia)
    tiempo_total = time.time() - tiempo_inicio
    print(f"Tiempo: {tiempo_total:.2f} segundos")

    # Mostrar resultados
    mostrar_planificacion(mejor_planificacion, matriz_actores_tomas)

---
# **Se aplica el algoritmo por fuerza bruta a matrices mas pequeñas que la del enunciado**

In [ ]:
matriz = [
    [1, 1, 0, 0],  # Toma 1: Actores 1 y 2
    [0, 1, 1, 0],  # Toma 2: Actores 2 y 3
    [1, 0, 1, 0],  # Toma 3: Actores 1 y 3
    [0, 0, 0, 1],  # Toma 4: Actor 4
]
max_tomas_por_dia = 3

ejecutar_algoritmo_fuerza_bruta(matriz, max_tomas_por_dia)
ejecutar_solver_pulp(matriz, max_tomas_por_dia)

ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA

Generando todas las asignaciones posibles...
Nueva mejor solucion encontrada - Costo: 4, Dias: 2, Asignacion: (0, 0, 0, 1)
Iteraciones Totales: 256, Iteraciones Validas: 252, Mejor costo: 4
Tiempo: 0.00 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===

DIA 1:
Tomas: [1, 2, 3]
Actores necesarios: [1, 2, 3]
Costo del dia: 3

DIA 2:
Tomas: [4]
Actores necesarios: [4]
Costo del dia: 1

COSTO TOTAL: 4
DIAS TOTALES: 2

ORGANIZAR SESIONES DE DOBLAJE - SOLVER PULP
Buscando solución entre 2 y 4 días...
Probando con 2 días...
Solución encontrada con 2 días. Costo: 4.0
Tiempo total: 0.05 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE ===

DIA: 1
  Tomas: [1, 2, 3]
  Actores necesarios: [1, 2, 3]
  Costo del dia: 3

DIA: 2
  Tomas: [4]
  Actores necesarios: [4]
  Costo del dia: 1

COSTO TOTAL: 4
DIAS TOTALES: 2



In [ ]:
matriz = [
[1, 1, 0],  # Toma 1: Actores 1, 2
[0, 1, 1],  # Toma 2: Actores 2, 3
[1, 0, 1],  # Toma 3: Actores 1, 3
[1, 1, 1]   # Toma 4: Actores 1, 2, 3
]
max_tomas_por_dia = 2

ejecutar_algoritmo_fuerza_bruta(matriz, max_tomas_por_dia)
ejecutar_solver_pulp(matriz, max_tomas_por_dia)

ORGANIZAR SESIONES DE DOBLAJE - FUERZA BRUTA

Generando todas las asignaciones posibles...
Nueva mejor solucion encontrada - Costo: 6, Dias: 2, Asignacion: (0, 0, 1, 1)
Iteraciones Totales: 256, Iteraciones Validas: 204, Mejor costo: 6
Tiempo: 0.00 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE POR FUERZA BRUTA ===

DIA 1:
Tomas: [1, 2]
Actores necesarios: [1, 2, 3]
Costo del dia: 3

DIA 2:
Tomas: [3, 4]
Actores necesarios: [1, 2, 3]
Costo del dia: 3

COSTO TOTAL: 6
DIAS TOTALES: 2

ORGANIZAR SESIONES DE DOBLAJE - SOLVER PULP
Buscando solución entre 2 y 4 días...
Probando con 2 días...
Solución encontrada con 2 días. Costo: 6.0
Tiempo total: 0.03 segundos

=== PLANIFICACION OPTIMA DE SESIONES DE DOBLAJE ===

DIA: 1
  Tomas: [1, 2]
  Actores necesarios: [1, 2, 3]
  Costo del dia: 3

DIA: 2
  Tomas: [3, 4]
  Actores necesarios: [1, 2, 3]
  Costo del dia: 3

COSTO TOTAL: 6
DIAS TOTALES: 2



---
# **Calcula la complejidad del algoritmo por fuerza bruta.**

#### 1. Generación de asignaciones (`generar_todas_asignaciones`)

   * Cada toma puede ir a cualquiera de los hasta **n** días posibles
   * Total de asignaciones:

     $$
       n^n
     $$
   * Coste:

     $$
       O(n^n)
     $$

#### 2. Conversión de asignación → planificación (`asignacion_a_planificacion`)

   * Recorrer las n tomas y agruparlas por día:

     $$
       O(n)
     $$
   * Comprobar en cada día que no haya más de k tomas: en el peor caso hay n días, y mirar el tamaño de cada lista es O(1), así que sigue siendo

     $$
       O(n)
     $$
   * Total por asignación:

     $$
       O(n)
     $$
   * En conjunto:

     $$
       O\bigl(n^n \times n\bigr) = O(n^{n+1})
     $$

#### 3. Cálculo de la función objetivo (`funcion_objetivo`)

   * Para cada día (hasta n días), recorres las tomas de ese día (en el peor caso n) y luego recorres el vector de actores (a elementos) para sumar únicos.
   * Coste aproximado por día:

     $$
       O(n + a) = O(n + a)
     $$
   * Para n días:

     $$
       O\bigl(n (n + a)\bigr) = O(n^2 + n\,a)
     $$
   * Total sobre todas las asignaciones válidas (que en el peor caso son casi todas):

     $$
       O\bigl(n^n \times (n^2 + n\,a)\bigr)
     \approx O\bigl(n^{n+2} + a\,n^{n+1}\bigr)
     $$

#### 4. Complejidad temporal combinada

Sumando los costes dominantes:

$$
  O(n^n) \;+\; O(n^{n+1}) \;+\; O(n^{n+2} + a\,n^{n+1})
  \quad=\quad O(n^{n+2} + a\,n^{n+1})
$$

Pero como $n^{n+2}$ domina a $n^{n+1}$, queda:

$$
  \boxed{O(n^{n+2})}
$$

En términos prácticos, el término exponencial $n^n$ es ya tan abrumador que los factores polinomiales adicionales apenas cambian la conclusión de que el algoritmo crece de forma exponencial.

---
# **Diseña un algoritmo que mejore la complejidad del algorito por fuerza bruta.**

In [ ]:
from copy import deepcopy
import time

class BusquedaTabu:
    def __init__(self, matriz_actores, max_tomas_por_dia):
        self.matriz = matriz_actores
        self.max_tomas = max_tomas_por_dia
        self.n_tomas = len(matriz_actores)
        self.n_actores = len(matriz_actores[0])

    def solucion_inicial_voraz(self):
        """Genera una solución inicial usando heurística voraz"""
        tomas_pendientes = set(range(self.n_tomas))
        calendario = []

        while tomas_pendientes:
            dia_actual = []
            actores_usados = set()

            for _ in range(self.max_tomas):
                if not tomas_pendientes:
                    break

                mejor_toma = max(
                    tomas_pendientes,
                    key=lambda t: len(actores_usados & {i for i, actor in enumerate(self.matriz[t]) if actor})
                )

                dia_actual.append(mejor_toma)
                tomas_pendientes.remove(mejor_toma)
                actores_usados.update(i for i, actor in enumerate(self.matriz[mejor_toma]) if actor)

            calendario.append(dia_actual)

        return calendario

    def calcular_costo(self, calendario):
        """Calcula el costo total del calendario"""
        return sum(
            len({actor for toma in dia for actor, presente in enumerate(self.matriz[toma]) if presente})
            for dia in calendario
        )

    def generar_vecinos(self, calendario):
        """Genera todos los vecinos mediante intercambios y traslados"""
        vecinos = []

        # Intercambios entre días
        for i in range(len(calendario)):
            for j in range(i + 1, len(calendario)):
                for toma_i in calendario[i]:
                    for toma_j in calendario[j]:
                        nuevo = deepcopy(calendario)
                        nuevo[i][nuevo[i].index(toma_i)] = toma_j
                        nuevo[j][nuevo[j].index(toma_j)] = toma_i
                        vecinos.append((nuevo, f"swap_{i}_{toma_i}_{j}_{toma_j}"))

        # Traslados entre días
        for i in range(len(calendario)):
            for j in range(len(calendario)):
                if i != j:
                    for toma in calendario[i]:
                        if len(calendario[j]) < self.max_tomas:
                            nuevo = deepcopy(calendario)
                            nuevo[i].remove(toma)
                            nuevo[j].append(toma)
                            nuevo = [dia for dia in nuevo if dia]  # Eliminar días vacíos
                            vecinos.append((nuevo, f"move_{i}_{toma}_{j}"))

        return vecinos

    def buscar(self, max_iter=500, tamano_tabu=20):
        """Algoritmo de búsqueda tabú"""
        print("="*60)
        print("ORGANIZAR SESIONES DE DOBLAJE - BÚSQUEDA TABÚ")
        print("="*60)

        t0 = time.time()

        actual = self.solucion_inicial_voraz()
        mejor = deepcopy(actual)
        lista_tabu = []
        costo_mejor = self.calcular_costo(mejor)

        print(f"Solución inicial (Voraz) - Costo: {costo_mejor}, Días: {len(mejor)}")

        for iteracion in range(max_iter):
            vecinos = self.generar_vecinos(actual)

            if not vecinos:
                break

            # Encontrar mejor vecino (tabú o no tabú)
            mejor_vecino, mejor_mov = min(vecinos, key=lambda x: self.calcular_costo(x[0]))
            mejor_costo = self.calcular_costo(mejor_vecino)

            # Si el mejor vecino es tabú, buscar el mejor no tabú
            if mejor_mov in lista_tabu and mejor_costo >= costo_mejor:
                vecinos_no_tabu = [(v, m) for v, m in vecinos if m not in lista_tabu]
                if vecinos_no_tabu:
                    mejor_vecino, mejor_mov = min(vecinos_no_tabu, key=lambda x: self.calcular_costo(x[0]))
                    mejor_costo = self.calcular_costo(mejor_vecino)
                else:
                    break

            actual = mejor_vecino
            lista_tabu.append(mejor_mov)
            if len(lista_tabu) > tamano_tabu:
                lista_tabu.pop(0)

            # Actualizar mejor solución
            if mejor_costo < costo_mejor:
                mejor = deepcopy(actual)
                costo_mejor = mejor_costo
                print(f"Iteración {iteracion+1}: Nueva mejor solución - Costo: {costo_mejor}, Días: {len(mejor)}")

        print(f"Tiempo total: {time.time() - t0:.2f} segundos\n")
        return mejor

    def mostrar_resultado(self, calendario):
        """Muestra el resultado final"""
        print("=== PLANIFICACIÓN ÓPTIMA DE SESIONES DE DOBLAJE ===\n")

        costo_total = 0
        for i, dia in enumerate(calendario, 1):
            actores_dia = {actor for toma in dia for actor, presente in enumerate(self.matriz[toma]) if presente}
            costo_dia = len(actores_dia)
            costo_total += costo_dia

            print(f"DÍA {i}:")
            print(f"   Tomas: {sorted(t + 1 for t in dia)}")
            print(f"   Actores necesarios: {sorted(a + 1 for a in actores_dia)}")
            print(f"   Costo del día: {costo_dia}\n")

        print(f"COSTO TOTAL: {costo_total}")
        print(f"DÍAS TOTALES: {len(calendario)}\n")

def ejecutar_busqueda_tabu(matriz, max_tomas_por_dia):
    algoritmo = BusquedaTabu(matriz, max_tomas_por_dia)
    mejor_calendario = algoritmo.buscar()
    algoritmo.mostrar_resultado(mejor_calendario)

---
# **Aplicar al algoritmo optimizado la matriz y restricciones del enunciado**

In [ ]:
ejecutar_busqueda_tabu(matriz_enunciado, max_tomas_por_dia_enunciado)

ORGANIZAR SESIONES DE DOBLAJE - BÚSQUEDA TABÚ
Solución inicial (Voraz) - Costo: 31, Días: 5
Iteración 1: Nueva mejor solución - Costo: 30, Días: 5
Iteración 15: Nueva mejor solución - Costo: 29, Días: 5
Iteración 41: Nueva mejor solución - Costo: 28, Días: 5
Iteración 49: Nueva mejor solución - Costo: 27, Días: 5
Tiempo total: 7.68 segundos

=== PLANIFICACIÓN ÓPTIMA DE SESIONES DE DOBLAJE ===

DÍA 1:
   Tomas: [1, 2, 12, 20, 22, 29]
   Actores necesarios: [1, 2, 3, 4, 5, 6]
   Costo del día: 6

DÍA 2:
   Tomas: [6, 7, 13, 16, 25, 27]
   Actores necesarios: [1, 2, 4, 5, 10]
   Costo del día: 5

DÍA 3:
   Tomas: [3, 4, 10, 11, 15, 26]
   Actores necesarios: [1, 2, 3, 5, 6, 7, 8, 9]
   Costo del día: 8

DÍA 4:
   Tomas: [14, 17, 18, 19, 23, 24]
   Actores necesarios: [1, 3, 6]
   Costo del día: 3

DÍA 5:
   Tomas: [5, 8, 9, 21, 28, 30]
   Actores necesarios: [1, 2, 4, 6, 8]
   Costo del día: 5

COSTO TOTAL: 27
DÍAS TOTALES: 5



### SE OBTIENE EL ÓPTIMO AL IGUAL QUE CON EL SOLVER Y EN MENOS TIEMPO.

---
# **¿Cuál es la estructura de datos que mejor se adapta al problema? Argumenta la respuesta.**

La estructura `List[List[int]]` es la más adecuada para este problema por las siguientes razones:

- Representación natural del dominio:
  - Se alinea directamente con el modelo conceptual: días que contienen tomas
  - Cada sublista representa un día de filmación con sus tomas correspondientes
  - Facilita la comprensión y visualización del calendario resultante
  - Mantiene la estructura temporal inherente al problema de planificación

- Eficiencia en operaciones críticas:
  - Cálculo de costo: Permite iteración eficiente día por día para contar actores únicos mediante `sum(len({actor for toma in dia...}))`
  - Generación de vecinos: Facilita intercambios directos entre días y traslados de tomas con operaciones simples como `remove()` y `append()`
  - Acceso indexado: Las operaciones `calendario[i]` y `calendario[j]` proporcionan acceso directo para modificaciones
  - Gestión de días vacíos: La línea `nuevo = [dia for dia in nuevo if dia]` permite eliminar días vacíos eficientemente

- Compatibilidad con búsqueda tabú:
  - Movimientos de intercambio: `swap_{i}_{toma_i}_{j}_{toma_j}` se implementa naturalmente intercambiando elementos entre sublistas
  - Movimientos de traslado: `move_{i}_{toma}_{j}` se ejecuta moviendo elementos entre sublistas diferentes
  - Identificación única: Los movimientos generan claves únicas para la lista tabú basadas en índices de días y tomas
  - Restricciones: Verifica fácilmente `len(calendario[j]) < self.max_tomas` para respetar límites diarios

- Ventajas sobre alternativas:
  - Versus diccionarios: Mejor para iteraciones secuenciales y operaciones de orden que requiere el algoritmo
  - Versus estructura plana: Encapsula mejor la lógica temporal y las restricciones por día
  - Versus sets: Mantiene el orden temporal y permite duplicados si fuera necesario
  - Versus tuplas: Proporciona mutabilidad necesaria para las operaciones de búsqueda local

- Consideraciones adicionales:
  - Memoria: Uso eficiente mediante `deepcopy()` solo cuando es necesario crear nuevas soluciones
  - Flexibilidad: Permite días con diferente número de tomas (hasta el máximo permitido)
  - Escalabilidad: Se adapta bien a instancias de diferentes tamaños sin cambios estructurales

---
# **Calcula la complejidad del algoritmo**

#### Variables principales

- `n`: cantidad total de tomas
- `m`: cantidad de actores
- `k`: máximo de tomas por día
- `d`: número de días (en general $d \approx \lceil n/k \rceil$)
- `I`: número máximo de iteraciones (`max_iter`)
- `V`: número de vecinos generados por iteración

#### 1. Costo por evaluación de una solución (calendario)

La función `calcular_costo` recorre:
- Cada día (hasta `d`)
- Cada toma dentro del día (hasta `k`)
- Cada actor dentro de la toma (hasta `m`)

Por lo tanto:
$$
\text{Complejidad por evaluación} = O(d \cdot k \cdot m) = O(n \cdot m)
$$

#### 2. Generación de vecinos

La función `generar_vecinos` considera dos tipos de movimientos:

##### Intercambios entre tomas de diferentes días:
- Pares de días: $O(d^2)$
- Por par de días, combinaciones de tomas: $k \times k = k^2$
- Vecinos generados: $O(d^2 \cdot k^2)$

##### Traslados de una toma de un día a otro:
- Días origen: $d$
- Días destino: $d$  
- Tomas en día origen: $k$
- Vecinos generados: $O(d^2 \cdot k)$

Total de vecinos:
$$V = O(d^2k^2 + d^2k) = O(d^2k^2)$$

#### 3. Coste por iteración

En cada iteración:

1. Generación de vecinos: cada vecino usa `deepcopy(calendario)` (coste $O(n)$) y hay $V=O(d^2k^2)$ vecinos:
   $$O(d^2k^2 \cdot n)$$

2. Evaluación de vecinos: cada uno con coste $O(n \cdot m)$:
   $$O(d^2k^2 \cdot n \cdot m)$$

El término dominante es:
$$O(d^2k^2 \cdot n \cdot m)$$

#### 4. Complejidad total de la Búsqueda Tabú

Con `I` iteraciones:
$$T(n,k,m,I) = O\bigl(I \cdot d^2 \cdot k^2 \cdot n \cdot m\bigr)$$

##### Simplificación usando $d \approx n/k$

$$d^2k^2 = \bigl(\tfrac{n}{k}\bigr)^2 k^2 = n^2$$

Por tanto:
$$T(n,k,m,I) = O\bigl(I \cdot n^2 \cdot n \cdot m\bigr) = O(I \cdot n^3 \cdot m)$$

#### 5. Casos particulares y dominancia asintótica

- Si `m` es constante o pequeño frente a `n` (caso típico con pocos actores), queda:
  $$O(I \cdot n^3)$$

- Si `m = O(n)`, entonces $O(I\cdot n^3\cdot m)=O(I\cdot n^4)$, pero esto sucede sólo si el número de actores crece al mismo ritmo que las tomas.

#### 6. Complejidad Final

Podemos resumir la complejidad del algoritmo Tabú según diferentes escenarios:

* Forma general (sin asumir relación entre parámetros):

  $$
  \boxed{T(n,m,I) = O(I \cdot n^3 \cdot m)}
  $$

* Caso práctico habitual (pocos actores, m ≪ n):

  $$
  \boxed{T(n,I) = O(I \cdot n^3)}
  $$

* Escenario extremo (cantidad de actores comparable a tomas, m = O(n)):

  $$
  \boxed{T(n,I) = O(I \cdot n^4)}
  $$


---
# **Argumenta porque crees que mejora el algoritmo por fuerza bruta.**

| Característica                | Fuerza bruta                                                                                      | Búsqueda Tabú                                                                                                                                         |
|-------------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| Escalabilidad                 | Explota todas las soluciones posibles → crecimiento super‑exponencial                             | Explora solo un subconjunto guiado (vecindario local) → crecimiento polinómico                                                                        |
| Control de tiempo de ejecución| No existe parámetro de parada: o termina tardísimo o no termina                                  | Se fija un máximo de iteraciones o presupuesto de tiempo, garantizando un resultado en un plazo predecible                                            |
| Calidad de la solución        | Garantiza óptimo global, pero inalcanzable para tamaños moderados                                 | Heurística que logra soluciones de muy buena calidad en tiempos prácticos, gracias a la lista tabú (evita ciclos) y criterio de aspiración            |
| Flexibilidad y adaptabilidad  | Difícil de extender con nuevas restricciones sin reescribir gran parte del enfoque                | Permite añadir restricciones (descansos, costes de cambio, etc.) y ajustar parámetros (tamaño de lista, iteraciones) sin cambiar la estructura básica |


La búsqueda tabú ofrece un compromiso óptimo entre:
- Tiempo de ejecución (polinómico y parametrizable)
- Calidad de las soluciones (muy cerca del óptimo)
- Flexibilidad para incluir nuevas restricciones


---
# Diseña un juego de datos de entrada aleatorio.

In [ ]:
import random

def generar_datos_aleatorios():
    # Configuración aleatoria
    tomas = random.randint(8, 15)
    actores = random.randint(5, 10)
    max_tomas_por_dia = random.randint(4, 10)

    print(f"Generando: {tomas} tomas, {actores} actores")

    # Generar matriz de actores
    matriz = []
    for i in range(tomas):
        fila = [random.randint(0, 1) for _ in range(actores)]
        # Asegurar que al menos un actor esté en cada toma
        if sum(fila) == 0:
            fila[random.randint(0, actores-1)] = 1
        matriz.append(fila)

    # Mostrar resultados
    print("matriz = [")
    for i, fila in enumerate(matriz):
        print(f"        {fila},  # Toma {i+1}")
    print("]")

    print(f"\nmax_tomas_por_dia = {max_tomas_por_dia}")

    total_iteraciones = total_posibilidades_sin_orden(len(matriz), max_tomas_por_dia)
    print(f"\nSe necesitan {total_iteraciones} iteraciones de fuerza bruta.")

    return matriz, max_tomas_por_dia


def generar_datos_y_ejecutar_ambos_algoritmos():
    matriz, max_tomas_por_dia = generar_datos_aleatorios()
    ejecutar_busqueda_tabu(matriz, max_tomas_por_dia)
    ejecutar_solver_pulp(matriz, max_tomas_por_dia)

---
# Aplica el algoritmo al juego de datos aleatorio generado

In [ ]:
generar_datos_y_ejecutar_ambos_algoritmos()

Generando: 12 tomas, 8 actores
matriz = [
        [1, 0, 1, 0, 1, 1, 1, 1],  # Toma 1
        [0, 0, 1, 1, 1, 0, 0, 1],  # Toma 2
        [1, 0, 1, 0, 0, 0, 1, 0],  # Toma 3
        [0, 1, 1, 0, 0, 0, 0, 1],  # Toma 4
        [1, 1, 1, 0, 1, 1, 1, 1],  # Toma 5
        [1, 1, 1, 0, 0, 1, 1, 0],  # Toma 6
        [0, 1, 0, 0, 1, 0, 0, 0],  # Toma 7
        [0, 1, 1, 0, 1, 0, 1, 0],  # Toma 8
        [1, 1, 0, 1, 1, 1, 0, 1],  # Toma 9
        [1, 0, 1, 1, 0, 0, 1, 1],  # Toma 10
        [0, 1, 1, 0, 0, 1, 1, 1],  # Toma 11
        [0, 1, 1, 1, 0, 1, 1, 0],  # Toma 12
]

max_tomas_por_dia = 10

Se necesitan 4213584 iteraciones de fuerza bruta.
ORGANIZAR SESIONES DE DOBLAJE - BÚSQUEDA TABÚ
Solución inicial (Voraz) - Costo: 12, Días: 2
Tiempo total: 0.33 segundos

=== PLANIFICACIÓN ÓPTIMA DE SESIONES DE DOBLAJE ===

DÍA 1:
   Tomas: [1, 2, 3, 5, 6, 8, 9, 10, 11, 12]
   Actores necesarios: [1, 2, 3, 4, 5, 6, 7, 8]
   Costo del día: 8

DÍA 2:
   Tomas: [4, 7]
   Actores necesarios: [2, 3, 5,

---
# Describe brevemente en unas líneas como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño.


## 1. Algoritmos híbridos y paralelos
- Búsqueda Tabú + Algoritmos Genéticos: Usar poblaciones de soluciones con operadores genéticos optimizados
- Paralelización: Evaluar múltiples vecindarios simultáneamente, especialmente crítico para instancias grandes (100+ tomas)
- Machine Learning: Entrenar modelos para predecir movimientos prometedores y reducir espacio de búsqueda

## 2. Variaciones realistas del problema
- Costos diferenciados: Actores con tarifas variables, penalizaciones por días no consecutivos
- Restricciones temporales: Disponibilidad limitada de actores, fechas de entrega
- Calidad de grabación: Minimizar cambios de configuración técnica entre tomas
- Recursos limitados: Estudios múltiples, equipos técnicos especializados

## 3. Problemas relacionados
- Scheduling cinematográfico: Locaciones, clima, disponibilidad de equipos
- Planificación de conciertos: Músicos, instrumentos, repertorio
- Organización de conferencias: Ponentes, salas, audiencias específicas

## 4. Métricas avanzadas
- Análisis de robustez: Sensibilidad ante cambios de última hora
- Equidad: Distribución balanceada de carga de trabajo entre actores
- Sostenibilidad: Minimizar desplazamientos y impacto ambiental

---
# Bibliografía utilizada

- CALDAS LIMA, Alberto. Aplicación de algoritmos heurísticos para optimizar el coste de doblaje de películas. A Coruña, 2014. Proyecto fin de máster. Universidad de A Coruña. Directores: Silvia María Lorenzo Freire, María Luisa Carpente Rodríguez. [En línea]. Disponible en: http://eio.usc.es/pub/mte/descargas/ProyectosFinMaster/Proyecto_759.pdf